In [1]:
import pandas as pd
import numpy as np

In [2]:
import psycopg2

print("Connecting to the PostgreSQL database...")

Connecting to the PostgreSQL database...


In [37]:
# Establish a connection to the PostgreSQL database
try:
    connection = psycopg2.connect(
        host="db.pqebyefuvarvwvwovchh.supabase.co",
        database="postgres",
        user="postgres",
        password="Hoangthinh@2004"
    )
    cursor = connection.cursor()
    print("Connection successful!")
except Exception as e:
    print(f"An error occurred: {e}")

Connection successful!


In [24]:
query = """
        SELECT table_name
        FROM information_schema.tables
        WHERE table_schema = 'public'
        """

try:
    cursor.execute(query)
    records = cursor.fetchall()
except (psycopg2.errors.InvalidTextRepresentation, psycopg2.errors.InFailedSqlTransaction) as e:
    print(f"An error occurred: {e}")
    connection.rollback()

    cursor.execute(query)
    records = cursor.fetchall()

df = pd.DataFrame(records, columns=[desc[0] for desc in cursor.description])
df.head(20)

,table_name
0,roles
1,users
2,restaurants
3,signature_foods
4,health_profiles
5,categories
6,tags
7,user_tags
8,restaurant_tags
9,posts


In [38]:
# Fetch data from the restaurants and related tables
cursor.execute("""
    select b.restaurant_id
        ,a.name as restaurant_name
        ,a.address
        ,c.category_id
        ,d.name as category_name
        ,b.tag_id
        ,c.name as tag_name
    from restaurants a
    inner join restaurant_tags b
        on a.id = b.restaurant_id
    inner join tags c
        on c.id = b.tag_id
    inner join categories d 
        on d.id = c.category_id
""")
records = cursor.fetchall() # Fetch all records from the executed query

# Create a DataFrame from the fetched records
restaurants = pd.DataFrame(records, columns=[desc[0] for desc in cursor.description])
restaurants.head(10)

,restaurant_id,restaurant_name,address,category_id,category_name,tag_id,tag_name
0,1,Pho Viet Budget,"Hanoi, Vietnam",1,Price Range,1,Budget
1,1,Pho Viet Budget,"Hanoi, Vietnam",2,Country,10,Vietnamese
2,1,Pho Viet Budget,"Hanoi, Vietnam",3,Food Type,22,Noodles
3,1,Pho Viet Budget,"Hanoi, Vietnam",4,Restaurant Style,30,Casual
4,1,Pho Viet Budget,"Hanoi, Vietnam",8,Decoration Style,71,Cozy
5,2,Tokyo Sushi Luxury,"Ho Chi Minh City, Vietnam",1,Price Range,3,Premium
6,2,Tokyo Sushi Luxury,"Ho Chi Minh City, Vietnam",2,Country,11,Japanese
7,2,Tokyo Sushi Luxury,"Ho Chi Minh City, Vietnam",3,Food Type,20,Seafood
8,2,Tokyo Sushi Luxury,"Ho Chi Minh City, Vietnam",4,Restaurant Style,31,Luxury
9,2,Tokyo Sushi Luxury,"Ho Chi Minh City, Vietnam",8,Decoration Style,70,Modern


In [39]:
# Create restaurant profile by aggregating tags

# Concat tag_name seperate by | restaurant_id
restaurants_profile = restaurants.groupby(["restaurant_id", "restaurant_name", "address"])["tag_name"] \
                                 .apply(lambda x: ' | '.join(x)) \
                                 .reset_index(name="restaurantCharacteristics")
restaurants_profile.head(10)

,restaurant_id,restaurant_name,address,restaurantCharacteristics
0,1,Pho Viet Budget,"Hanoi, Vietnam",Budget | Vietnamese | Noodles | Casual | Cozy
1,2,Tokyo Sushi Luxury,"Ho Chi Minh City, Vietnam",Premium | Japanese | Seafood | Luxury | Modern
2,3,Da Nang Vegan Cozy,"Da Nang, Vietnam",MidRange | Vegan | Cozy | Healthy | Themed
3,4,Hanoi Steak House,"Hanoi, Vietnam",MidRange | Vietnamese | Steak | Luxury | Modern
4,5,Saigon Ramen Buffet,"Ho Chi Minh City, Vietnam",Budget | Japanese | Ramen | Buffet | Casual
5,6,Korean BBQ Midrange,"Seoul, Korea",MidRange | Korean | Seafood | Casual | Cozy
6,7,French Fine Dining,"Paris, France",Premium | Korean | Healthy | Luxury | Modern
7,8,Italian Cozy Pasta,"Rome, Italy",MidRange | Korean | Noodles | Cozy | Themed
8,9,Thai Spicy Street Food,"Bangkok, Thailand",Budget | Korean | Spicy | Casual | Themed
9,10,Mexican Themed Cantina,"Mexico City, Mexico",MidRange | Korean | Spicy | Buffet | Themed


In [10]:
connection.rollback()

In [40]:
# Fetch data from the users and related tables
cursor.execute("""
select b.user_id
      ,a.username
      ,c.category_id
      ,d.name as category_name
      ,b.tag_id
      ,c.name as tag_name
from users a
inner join user_tags b
  on b.user_id = a.id
inner join tags c
    on c.id = b.tag_id
inner join categories d 
    on d.id = c.category_id
""")
records = cursor.fetchall() # Fetch all records from the executed query

# Create a DataFrame from the fetched records
users = pd.DataFrame(records, columns=[desc[0] for desc in cursor.description])
users.head(10)

,user_id,username,category_id,category_name,tag_id,tag_name
0,1,alice,2,Country,10,Vietnamese
1,1,alice,8,Decoration Style,71,Cozy
2,1,alice,1,Price Range,1,Budget
3,2,bob,2,Country,11,Japanese
4,2,bob,4,Restaurant Style,31,Luxury
5,2,bob,3,Food Type,20,Seafood
6,3,charlie,3,Food Type,21,Vegan
7,3,charlie,5,Food Characteristics,41,Healthy
8,3,charlie,1,Price Range,2,MidRange


In [41]:
# create user profile by aggregating tags
# Concat tag_name seperate by | user_id
users_profile = users.groupby(["user_id","username"])["tag_name"] \
                     .apply(lambda x: ' | '.join(x)) \
                     .reset_index(name="userCharacteristics")
users_profile.head(10)

,user_id,username,userCharacteristics
0,1,alice,Vietnamese | Cozy | Budget
1,2,bob,Japanese | Luxury | Seafood
2,3,charlie,Vegan | Healthy | MidRange


In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [49]:
# create user_metrics by choose userID and userCharacteristics
user_metrics = users_profile[["user_id","userCharacteristics"]]


In [50]:
# create restaurant_metrics by choose restaurantID and restaurantCharacteristics
restaurant_metrics = restaurants_profile[["restaurant_id","restaurantCharacteristics"]]

In [55]:
# create vectorizer
vectorizer = TfidfVectorizer()

user_vecs = vectorizer.fit_transform(users_profile["userCharacteristics"])
restaurant_vecs = vectorizer.transform(restaurants_profile["restaurantCharacteristics"])

# similarity between userCharacteristics and restaurantCharacteristics
similarity = cosine_similarity(user_vecs, restaurant_vecs)
print(similarity)


[[1.         0.         0.28867513 0.33333333 0.40824829 0.33333333
  0.         0.40824829 0.57735027 0.        ]
 [0.         1.         0.         0.33333333 0.40824829 0.33333333
  0.40824829 0.         0.         0.        ]
 [0.         0.         0.8660254  0.33333333 0.         0.33333333
  0.40824829 0.40824829 0.         0.57735027]]


In [56]:
# create final result 
df_result = pd.DataFrame(similarity
                       , index=users_profile["user_id"]
                       , columns=restaurants_profile["restaurant_id"])
df_long = df_result.reset_index() \
                   .melt(id_vars="user_id", var_name="restaurant_id", value_name="similarity_score")
df_long = df_long.sort_values(by=["user_id", "similarity_score"], ascending=[True, False])
df_long.head(20)

,user_id,restaurant_id,similarity_score
0,1,1,1.000000
24,1,9,0.577350
12,1,5,0.408248
21,1,8,0.408248
9,1,4,0.333333
15,1,6,0.333333
6,1,3,0.288675
3,1,2,0.000000
18,1,7,0.000000
27,1,10,0.000000


In [58]:
# match user_id and restaurant_id with users & restaurants df
df_final = df_long.merge(users_profile[["user_id","username"]], on="user_id", how="left") \
                  .merge(restaurants_profile[["restaurant_id","restaurant_name","address"]], on="restaurant_id", how="left")
df_final.head(20)

,user_id,restaurant_id,similarity_score,username,restaurant_name,address
0,1,1,1.000000,alice,Pho Viet Budget,"Hanoi, Vietnam"
1,1,9,0.577350,alice,Thai Spicy Street Food,"Bangkok, Thailand"
2,1,5,0.408248,alice,Saigon Ramen Buffet,"Ho Chi Minh City, Vietnam"
3,1,8,0.408248,alice,Italian Cozy Pasta,"Rome, Italy"
4,1,4,0.333333,alice,Hanoi Steak House,"Hanoi, Vietnam"
5,1,6,0.333333,alice,Korean BBQ Midrange,"Seoul, Korea"
6,1,3,0.288675,alice,Da Nang Vegan Cozy,"Da Nang, Vietnam"
7,1,2,0.000000,alice,Tokyo Sushi Luxury,"Ho Chi Minh City, Vietnam"
8,1,7,0.000000,alice,French Fine Dining,"Paris, France"
9,1,10,0.000000,alice,Mexican Themed Cantina,"Mexico City, Mexico"
